In [1]:
import numpy as np

In [ ]:
def CG(A, b, x0):
    x = [x0]
    r = np.matmul(A,x) - b
    p = -r
    k = 0
    while r[-1] != 0:
        a.append(np.matmul(np.transpose(r[-1]),r[-1])
                 /np.matmul(np.matmul(np.transpose(p[-1]),A),p[-1]))
        x.append(x[])